In [1]:
import string

%pip install nltk
%pip install pandas
%pip install scikit-learn

import pandas as pd
import nltk
from nltk.corpus import wordnet

nltk.download('punkt')


from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import numpy as np
import re
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')




[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to /home/imranm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/imranm3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/imranm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/imranm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/imranm3/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Pre-processing

In [2]:
lemmatizer = WordNetLemmatizer()

sw = stopwords.words('english')
porter_stemmer = PorterStemmer()


def text_cleaning(text):
    text = str (text)

    printable = set(string.printable)
    text = ''.join(filter(lambda x: x in printable, text))
    text = text.replace('\x00', ' ')  # remove nulls
    text = text.replace('\r', ' ')
    text = text.replace('\n', ' ')
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    text = text.lower()  # Lowercasing

    words = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    wn_tags = {'N': wordnet.NOUN, 'V': wordnet.VERB, 'R': wordnet.ADV, 'J': wordnet.ADJ}
    # lemmatize each word with its POS tag
    lemmatized_words = []
    for word, pos in pos_tags:
        if pos[0] in wn_tags:
            wn_tag = wn_tags[pos[0]]
            lemmatized_words.append(lemmatizer.lemmatize(word, wn_tag))
        else:
            lemmatized_words.append(word)
    text = ' '.join([str(elem) for elem in lemmatized_words])

    words= text.split()
    stemmed_words=[porter_stemmer.stem(word=word) for word in words] # Stemming
    text = ' '.join([str(elem) for elem in stemmed_words])

    text = text.strip()
    return text


In [3]:
text_cleaning("I'm getting the same error, on 2.9.0 but I reproduced it in 2.8.0 and 2.9.1 too.")

'i m get the same error on 2 9 0 but i reproduc it in 2 8 0 and 2 9 1 too'

In [4]:
df = pd.read_csv("anger_causes.csv")

In [5]:
comments = df["Comment"]
comments

0       @synandi Tensorflow 2.12.0 Can't use GPU on Wi...
1       Sorry, even with the addition of `prefetch` I ...
2       @zyxkad Your problem is not related to the one...
3       I am using Teachable Machine for my audio mode...
4       @tiruk007 Having the same issue. CUDA 11.8 and...
                              ...                        
2606    @sanatmpa1 ,\nI was able to reproduce the issu...
2607    This issue is still reproducible on tf nightly...
2608    @jvishnuvardhan ,\nI was able to reproduce the...
2609    TF 2.11 raises `ValueError: num_tokens must be...
2610    Mine was not a solution it was just to check t...
Name: Comment, Length: 2611, dtype: object

In [6]:
corpus = df["Emotion Causes"]
corpus

0                            Can't use GPU on Windows 11?
1                     I am still seeing this log message.
2       "Your problem is not related to the one that t...
3       I am using Teachable Machine for my audio mode...
4                                  Having the same issue.
                              ...                        
2606    I was able to reproduce the issue in tf v2.6 a...
2607    This issue is still reproducible on tf nightly...
2608    I was able to reproduce the issue in tensorflo...
2609    TF 2.11 raises ValueError: num_tokens must be ...
2610    "mine was not a solution it was just to check ...
Name: Emotion Causes, Length: 2611, dtype: object

In [7]:
processed_corpus = corpus.apply(text_cleaning)
processed_corpus

0                              can t use gpu on window 11
1                           i be still see thi log messag
2       your problem be not relat to the one that thi ...
3       i be use teachabl machin for my audio model i ...
4                                      have the same issu
                              ...                        
2606    i be abl to reproduc the issu in tf v2 6 and n...
2607    thi issu be still reproduc on tf nightli 2 13 ...
2608    i be abl to reproduc the issu in tensorflow 2 ...
2609    tf 2 11 rais valueerror num token must be set ...
2610    mine be not a solut it be just to check that w...
Name: Emotion Causes, Length: 2611, dtype: object

In [8]:
vectorizer = TfidfVectorizer(stop_words="english",ngram_range=(1,1))
X = vectorizer.fit_transform(processed_corpus)

In [9]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X.toarray())

In [10]:
len(X_norm)

2611

## Clustering (DBSCAN)

In [11]:
db = DBSCAN(metric='cosine', min_samples=3, eps=0.45).fit(X)
labels = db.labels_

no_clusters = len(np.unique(labels) )
no_noise = np.sum(np.array(labels) == -1, axis=0)

print('Estimated num of clusters: %d' % no_clusters)
print('Estimated num of noise points: %d' % no_noise)

Estimated num of clusters: 63
Estimated num of noise points: 2147


In [12]:
print(db.labels_)
unique, counts = np.unique(db.labels_, return_counts = True)
print(dict(zip(unique, counts)))

[33  0 -1 ... 22 -1 -1]
{-1: 2147, 0: 7, 1: 142, 2: 3, 3: 3, 4: 4, 5: 12, 6: 14, 7: 7, 8: 12, 9: 4, 10: 4, 11: 4, 12: 3, 13: 5, 14: 18, 15: 6, 16: 3, 17: 3, 18: 3, 19: 4, 20: 3, 21: 3, 22: 9, 23: 6, 24: 5, 25: 8, 26: 6, 27: 4, 28: 17, 29: 4, 30: 9, 31: 7, 32: 3, 33: 16, 34: 6, 35: 3, 36: 3, 37: 3, 38: 3, 39: 3, 40: 3, 41: 4, 42: 4, 43: 4, 44: 5, 45: 5, 46: 3, 47: 7, 48: 3, 49: 3, 50: 3, 51: 3, 52: 3, 53: 5, 54: 5, 55: 3, 56: 3, 57: 3, 58: 3, 59: 3, 60: 4, 61: 3}


In [13]:
cluster_freq = dict(zip(unique, counts))
print(type(cluster_freq))
print(sorted(dict(zip(unique, counts)).items(), key=lambda x:x[1], reverse=True))

<class 'dict'>
[(-1, 2147), (1, 142), (14, 18), (28, 17), (33, 16), (6, 14), (5, 12), (8, 12), (22, 9), (30, 9), (25, 8), (0, 7), (7, 7), (31, 7), (47, 7), (15, 6), (23, 6), (26, 6), (34, 6), (13, 5), (24, 5), (44, 5), (45, 5), (53, 5), (54, 5), (4, 4), (9, 4), (10, 4), (11, 4), (19, 4), (27, 4), (29, 4), (41, 4), (42, 4), (43, 4), (60, 4), (2, 3), (3, 3), (12, 3), (16, 3), (17, 3), (18, 3), (20, 3), (21, 3), (32, 3), (35, 3), (36, 3), (37, 3), (38, 3), (39, 3), (40, 3), (46, 3), (48, 3), (49, 3), (50, 3), (51, 3), (52, 3), (55, 3), (56, 3), (57, 3), (58, 3), (59, 3), (61, 3)]


In [14]:
# Obtain text points for each cluster
text_points = []
unique_labels = set(db.labels_)
cluster_ids = []

for label in unique_labels:
    if cluster_freq[label] < 10 or label == -1:
        continue
    cluster_indices = np.where(db.labels_ == label)[0]
    cluster_docs = [corpus[j] for j in cluster_indices]
    cluster_vecs = X[cluster_indices]
    centroid_vec = np.mean(cluster_vecs.toarray(), axis=0)
    similarity_scores = cosine_similarity(cluster_vecs, [centroid_vec])
    # print(cluster_indices)
    text_point_index = cluster_indices[np.argmax(similarity_scores)]
    text_points.append(corpus[text_point_index])
    print(f"Cluster #{label} text point: {corpus[text_point_index]}")

    for idx in cluster_indices:
        cluster_ids.append([label, comments[idx], corpus[idx]])
        # print(idx, corpus[idx])


Cluster #1 text point: I am also having this issue.
Cluster #5 text point: "Update file>" commit message.
Cluster #6 text point: No response.
Cluster #8 text point: "Resolve merge conflict?"
Cluster #14 text point: "still not working"
Cluster #28 text point: "In this case, it broke an internal test."
Cluster #33 text point: The windows build was still failed.


In [15]:
import pandas as pd 
import numpy as np

arr = np.asarray(cluster_ids)
pd.DataFrame(arr).to_csv('clusters.csv', index_label = "Index")    

## Top features

In [16]:
from collections import Counter

def remove_sw(word_list):
    keep = []
    for word in word_list:
        if not word in sw:
            keep.append(word)
    return keep


# Get the top features for each cluster
unique_labels = set(db.labels_)
top_n = 5 # Number of top features to retrieve for each cluster
feature_names = np.array(vectorizer.get_feature_names_out())
for label in unique_labels:
    if cluster_freq[label] < 10 or label == -1:
        continue
    indices = np.where(db.labels_ == label)[0]
    indices = indices.tolist()
    sentences = []
    for idx in indices:
        sentences.append(processed_corpus[idx])
    words = []

    for sentence in sentences:
        # Convert the sentence to lowercase and split it into words
        words += sentence.lower().split()
    # words = remove_sw(words)
    # Count the frequency of each word using the Counter class
    word_counts = Counter(words)

    # Get the top 5 most common words
    top_words = word_counts.most_common(15)

    print(top_words)

    # cluster_vecs = X[indices].toarray()
    # centroid_vec = np.mean(cluster_vecs, axis=0)
    # # print(np.max(centroid_vec))
    # # print(np.sort(centroid_vec))
    # centroid_vec = np.argsort(centroid_vec)
    # # print(centroid_vec)
    # top_features_indices = centroid_vec[::-1][:top_n]
    # top_features = feature_names[top_features_indices]
    # # print(top_features)
    # print(f"Cluster #{label} top features: {', '.join(top_features)}")


[('i', 80), ('the', 77), ('be', 53), ('issu', 52), ('same', 51), ('have', 36), ('thi', 33), ('2', 29), ('with', 26), ('it', 25), ('still', 25), ('problem', 25), ('0', 25), ('to', 24), ('error', 22)]
[('updat', 9), ('commit', 8), ('the', 6), ('messag', 6), ('file', 5), ('at', 4), ('a', 4), ('it', 3), ('pleas', 3), ('use', 3), ('and', 3), ('pr', 3), ('still', 3), ('these', 2), ('make', 2)]
[('respons', 8), ('merg', 7), ('pr', 4), ('no', 4), ('the', 4), ('await', 3), ('be', 3), ('i', 3), ('thi', 2), ('delay', 2), ('my', 2), ('review', 2), ('and', 2), ('there', 2), ('it', 2)]
[('resolv', 10), ('conflict', 9), ('merg', 5), ('have', 4), ('i', 4), ('be', 4), ('to', 4), ('you', 3), ('intern', 3), ('not', 3), ('can', 2), ('pleas', 2), ('work', 2), ('issu', 2), ('thi', 2)]
[('work', 13), ('still', 7), ('not', 6), ('doesn', 6), ('t', 6), ('look', 5), ('it', 4), ('be', 3), ('like', 3), ('thi', 3), ('issu', 2), ('i', 2), ('test', 2), ('tf', 2), ('to', 2)]
[('be', 10), ('test', 9), ('fail', 8), ('br